In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from keras import models,backend,optimizers,layers
from keras.models import Model
from keras.layers import LSTM,Dense,Dropout,concatenate,Input,GRU,SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
import sys
import sklearn.externals 
import joblib
from keras.callbacks import EarlyStopping
from math import sqrt
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
def mae_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum(abs(y_v[:,d]-p_v[:,d]))/len(y_v)

def mse_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)

    return sum((y_v[:,d]-p_v[:,d])*(y_v[:,d]-p_v[:,d]))/len(y_v)

def mape_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)
    return (1/len(y_v))*(sum(abs((p_v[:,d]-y_v[:,d])/y_v[:,d])))

def rmse_f( y_v , p_v , d ):
    y_v[:,d]=y_v[:,d].astype(float)
    return sqrt(sum((y_v[:,d]-p_v[:,d])*(y_v[:,d]-p_v[:,d]))/len(y_v))
#data_loc= pd.read_csv('new0.csv', encoding='utf-8')

#data_loc
data= pd.read_csv('/content/drive/MyDrive/finaldata.csv', encoding='utf-8')
data

,timestamp,braden,ulcer_head,ulcer_shoulder,ulcer_arm,ulcer_buttocks,ulcer_leg,ulcer_heel,EMA_head,EMA_shoulder,...,movement_score_arm,movement_score_buttocks,movement_score_leg,movement_score_heel,outcome_head,outcome_shoulder,outcome_arm,outcome_buttocks,outcome_leg,outcome_heel
0,2022-07-20 00:00:00,19,0,1,1,0,1,0,7,88,...,0,5,2,3,109,110,1,146,93,54
1,2022-07-20 00:00:03,19,0,1,1,0,1,0,85,96,...,5,4,4,5,173,75,8,122,82,62
2,2022-07-20 00:00:06,19,0,1,1,0,1,0,58,12,...,1,5,4,1,178,164,13,165,178,104
3,2022-07-20 00:00:09,19,0,1,1,0,1,0,31,14,...,5,5,3,1,87,96,132,63,121,143
4,2022-07-20 00:00:12,19,0,1,1,0,1,0,100,20,...,2,4,3,5,159,29,37,53,83,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,2022-07-20 00:59:45,16,1,1,1,1,0,1,6,38,...,4,3,5,4,94,37,47,147,22,125
23996,2022-07-20 00:59:48,16,1,1,1,1,0,1,83,99,...,5,5,5,0,115,50,77,69,4,65
23997,2022-07-20 00:59:51,16,1,1,1,1,0,1,82,75,...,4,0,4,3,64,58,137,158,47,41
23998,2022-07-20 00:59:54,16,1,1,1,1,0,1,99,83,...,3,0,1,3,86,96,163,38,80,83


In [17]:
#data = data.drop(['timestamp'], axis=1)
loc_0_x=data.iloc[:,:19]
loc_0_x=loc_0_x.values
StandardScaler_scaler = preprocessing.StandardScaler()
StandardScaler_scaler = StandardScaler_scaler.fit(loc_0_x) #Stander
loc_0_x = StandardScaler_scaler.transform(loc_0_x)

loc_0_x=loc_0_x.reshape(len(loc_0_x),1,19) 
loc_0_y=data.iloc[:,19:].values
loc_0_y

array([[109, 110,   1, 146,  93,  54],
       [173,  75,   8, 122,  82,  62],
       [178, 164,  13, 165, 178, 104],
       ...,
       [ 64,  58, 137, 158,  47,  41],
       [ 86,  96, 163,  38,  80,  83],
       [ 29, 151,  59,  49, 126, 106]])

In [16]:



validation_sample=20000

#=============testing==============#
x_v=loc_0_x[validation_sample:]
y_v=loc_0_y[validation_sample:]

#=============training===============#
x=loc_0_x[:validation_sample]
y=loc_0_y[:validation_sample]

#training and testing

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

  

#build LSTM model
input_data=Input(shape=(1,19))
m=Bidirectional(LSTM(256,return_sequences=True))(input_data)
m=Dropout(0.3)(m)
m=LSTM(256)(m)
m=Dropout(0.3)(m)
#m=LSTM(256)(m)
#m=Dropout(0.2)(m)

m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
output=Dense(6,activation=None)(m)

model = Model(inputs=input_data,outputs=output)
callback = EarlyStopping(monitor="val_loss", patience=100, verbose=1, mode="auto")
model.compile(optimizer='adam',loss=['mse'],metrics=['mae','mape'])
result = model.fit(X_train,y_train,epochs=1500 ,batch_size=2400,validation_data=(X_test, y_test),callbacks=[callback])
#model.save('LSTM_model.h5')


Epoch 1/1500
6/6 [==============================] - 13s 954ms/step - loss: 10735.7490 - mae: 89.4214 - mape: 2919852.2500 - val_loss: 10405.1230 - val_mae: 87.5354 - val_mape: 14118715.0000
Epoch 2/1500
6/6 [==============================] - 4s 611ms/step - loss: 9155.6494 - mae: 80.6837 - mape: 56723464.0000 - val_loss: 6073.3169 - val_mae: 63.7000 - val_mape: 179801376.0000
Epoch 3/1500
6/6 [==============================] - 4s 616ms/step - loss: 4287.4302 - mae: 53.8907 - mape: 422535744.0000 - val_loss: 3998.7959 - val_mae: 52.2699 - val_mape: 685259776.0000
Epoch 4/1500
6/6 [==============================] - 4s 609ms/step - loss: 3433.6311 - mae: 49.1348 - mape: 548913280.0000 - val_loss: 3195.5210 - val_mae: 47.9504 - val_mape: 383219936.0000
Epoch 5/1500
4/6 [===================>..........] - ETA: 1s - loss: 3411.2295 - mae: 49.0920 - mape: 383961120.0000

KeyboardInterrupt: ignored

In [ ]:
score=model.evaluate(x_v,y_v)
print("MSE:","%.4f"%score[0]," / MAE:","%.4f"%score[1],"/ MAPE:","%.4f"%score[2])

In [ ]:
a=model.predict(x_v)
print("LSTM")
print("========MSE========")
print("1hr:%.4f\n"%mse_f(y_v,a,0),"2hrs:%.4f\n"%mse_f(y_v,a,1),"3hrs:%.4f\n"%mse_f(y_v,a,2),"4hrs:%.4f\n"%mse_f(y_v,a,3),"5hrs:%.4f\n"%mse_f(y_v,a,4),"6hrs:%.4f\n"%mse_f(y_v,a,5),"7hrs:%.4f\n"%mse_f(y_v,a,6),"8hrs:%.4f"%mse_f(y_v,a,7))
print("========MAE========")
print("1hr:%.4f\n"%mae_f(y_v,a,0),"2hrs:%.4f\n"%mae_f(y_v,a,1),"3hrs:%.4f\n"%mae_f(y_v,a,2),"4hrs:%.4f\n"%mae_f(y_v,a,3),"5hrs:%.4f\n"%mae_f(y_v,a,4),"6hrs:%.4f\n"%mae_f(y_v,a,5),"7hrs:%.4f\n"%mae_f(y_v,a,6),"8hrs:%.4f"%mae_f(y_v,a,7))
print("========MAPE========")
print("1hr:%.4f\n"%mape_f(y_v,a,0),"2hrs:%.4f\n"%mape_f(y_v,a,1),"3hrs:%.4f\n"%mape_f(y_v,a,2),"4hrs:%.4f\n"%mape_f(y_v,a,3),"5hrs:%.4f\n"%mape_f(y_v,a,4),"6hrs:%.4f\n"%mape_f(y_v,a,5),"7hrs:%.4f\n"%mape_f(y_v,a,6),"8hrs:%.4f"%mape_f(y_v,a,7))
print("========RMSE========")
print("1hr:%.4f\n"%rmse_f(y_v,a,0),"2hrs:%.4f\n"%rmse_f(y_v,a,1),"3hrs:%.4f\n"%rmse_f(y_v,a,2),"4hrs:%.4f\n"%rmse_f(y_v,a,3),"5hrs:%.4f\n"%rmse_f(y_v,a,4),"6hrs:%.4f\n"%rmse_f(y_v,a,5),"7hrs:%.4f\n"%rmse_f(y_v,a,6),"8hrs:%.4f"%rmse_f(y_v,a,7))


In [ ]:
#build GRU model
input_data=Input(shape=(1,19))
m=GRU(256,return_sequences=True)(input_data)
m=Dropout(0.3)(m)
m=GRU(256)(m)
m=Dropout(0.3)(m)
#m=LSTM(256)(m)
#m=Dropout(0.2)(m)

m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
m=Dense(256,activation='relu')(m)
m=Dropout(0.3)(m)
output=Dense(8,activation=None)(m)

model = Model(inputs=input_data,outputs=output)
callback = EarlyStopping(monitor="val_loss", patience=100, verbose=1, mode="auto")
model.compile(optimizer='adam',loss=['mse'],metrics=['mae','mape'])
result = model.fit(X_train,y_train,epochs=1500 ,batch_size=2400,validation_data=(X_test, y_test),callbacks=[callback])
#model.save('GRU_model.h5')

In [ ]:
a=model.predict(x_v)
print("GRU")
print("========MSE========")
print("1hr:%.4f\n"%mse_f(y_v,a,0),"2hrs:%.4f\n"%mse_f(y_v,a,1),"3hrs:%.4f\n"%mse_f(y_v,a,2),"4hrs:%.4f\n"%mse_f(y_v,a,3),"5hrs:%.4f\n"%mse_f(y_v,a,4),"6hrs:%.4f\n"%mse_f(y_v,a,5),"7hrs:%.4f\n"%mse_f(y_v,a,6),"8hrs:%.4f"%mse_f(y_v,a,7))
print("========MAE========")
print("1hr:%.4f\n"%mae_f(y_v,a,0),"2hrs:%.4f\n"%mae_f(y_v,a,1),"3hrs:%.4f\n"%mae_f(y_v,a,2),"4hrs:%.4f\n"%mae_f(y_v,a,3),"5hrs:%.4f\n"%mae_f(y_v,a,4),"6hrs:%.4f\n"%mae_f(y_v,a,5),"7hrs:%.4f\n"%mae_f(y_v,a,6),"8hrs:%.4f"%mae_f(y_v,a,7))
print("========MAPE========")
print("1hr:%.4f\n"%mape_f(y_v,a,0),"2hrs:%.4f\n"%mape_f(y_v,a,1),"3hrs:%.4f\n"%mape_f(y_v,a,2),"4hrs:%.4f\n"%mape_f(y_v,a,3),"5hrs:%.4f\n"%mape_f(y_v,a,4),"6hrs:%.4f\n"%mape_f(y_v,a,5),"7hrs:%.4f\n"%mape_f(y_v,a,6),"8hrs:%.4f"%mape_f(y_v,a,7))
print("========RMSE========")
print("1hr:%.4f\n"%rmse_f(y_v,a,0),"2hrs:%.4f\n"%rmse_f(y_v,a,1),"3hrs:%.4f\n"%rmse_f(y_v,a,2),"4hrs:%.4f\n"%rmse_f(y_v,a,3),"5hrs:%.4f\n"%rmse_f(y_v,a,4),"6hrs:%.4f\n"%rmse_f(y_v,a,5),"7hrs:%.4f\n"%rmse_f(y_v,a,6),"8hrs:%.4f"%rmse_f(y_v,a,7))
